In [1]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from tensorflow.python.keras.callbacks import TensorBoard
import pickle
import time

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0

dense_layers = [0]
layer_sizes = [64]
conv_layers = [3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'],
                          )

            model.fit(X, y,
                      batch_size=32,
                      epochs=10,
                      validation_split=0.3,
                      callbacks=[tensorboard])

model.save('64x3-CNN.model')

3-conv-64-nodes-0-dense-1550675512
Train on 1382 samples, validate on 593 samples
Epoch 1/10
1382/1382 [==============================] - 10s 8ms/step - loss: 0.6945 - acc: 0.4913 - val_loss: 0.6992 - val_acc: 0.4722
Epoch 2/10
1382/1382 [==============================] - 10s 7ms/step - loss: 0.6904 - acc: 0.5130 - val_loss: 0.6904 - val_acc: 0.5211
Epoch 3/10
1382/1382 [==============================] - 9s 7ms/step - loss: 0.6804 - acc: 0.5557 - val_loss: 0.6880 - val_acc: 0.5177
Epoch 4/10
1382/1382 [==============================] - 10s 7ms/step - loss: 0.6562 - acc: 0.6266 - val_loss: 0.7028 - val_acc: 0.5531
Epoch 5/10
1382/1382 [==============================] - 10s 7ms/step - loss: 0.6205 - acc: 0.6650 - val_loss: 0.6574 - val_acc: 0.6391
Epoch 6/10
1382/1382 [==============================] - 10s 7ms/step - loss: 0.5931 - acc: 0.6823 - val_loss: 0.6496 - val_acc: 0.6560
Epoch 7/10
1382/1382 [==============================] - 10s 7ms/step - loss: 0.5716 - acc: 0.6997 - val_loss:

In [3]:
import cv2
import tensorflow as tf

CATEGORIES = ["Dog", "Cat"]  # will use this to convert prediction num to string value

def prepare(filepath):
    IMG_SIZE = 50  # 50 in txt-based
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)  # read in the image, convert to grayscale
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)  # return the image with shaping that TF wants.

model = tf.keras.models.load_model("64x3-CNN.model")
prediction = model.predict([prepare('dog.jpg')])  # REMEMBER YOU'RE PASSING A LIST OF THINGS YOU WISH TO PREDICT
print(CATEGORIES[int(prediction[0][0])])

Cat
